![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_MPNet.ipynb)

# Import ONNX MPNet models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support for this annotator was introduced in `Spark NLP 5.1.1`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for MPNet from HuggingFace and they have to be in `Sentence Similarity` category. Meaning, you cannot use MPNet models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum==1.24.0 onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the MPNet model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import MPNetTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = MPNetTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

('onnx_models/sentence-transformers/all-mpnet-base-v2/tokenizer_config.json',
 'onnx_models/sentence-transformers/all-mpnet-base-v2/special_tokens_map.json',
 'onnx_models/sentence-transformers/all-mpnet-base-v2/vocab.txt',
 'onnx_models/sentence-transformers/all-mpnet-base-v2/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 426552
-rw-r--r-- 1 root root       591 Jun 15 23:00 config.json
-rw-r--r-- 1 root root 435826548 Jun 15 23:00 model.onnx
-rw-r--r-- 1 root root       964 Jun 15 23:00 special_tokens_map.json
-rw-r--r-- 1 root root      1475 Jun 15 23:00 tokenizer_config.json
-rw-r--r-- 1 root root    710944 Jun 15 23:00 tokenizer.json
-rw-r--r-- 1 root root    231536 Jun 15 23:00 vocab.txt


- Now we need to move `vocab.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find it.

In [4]:
!mkdir -p {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [5]:
!ls -l {EXPORT_PATH}/assets

total 228
-rw-r--r-- 1 root root 231536 Jun 15 23:00 vocab.txt


Voila! We have our `vocab.txt` inside assets directory

## Import and Save MPNet in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [ ]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `MPNetEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `MPNetEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [8]:
from sparknlp.annotator import MPNetEmbeddings

MPNet = MPNetEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document"])\
    .setOutputCol("MPNet")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
MPNet.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX MPNet model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 425692
drwxr-xr-x 3 root root      4096 Jun 15 23:03 fields
drwxr-xr-x 2 root root      4096 Jun 15 23:03 metadata
-rw-r--r-- 1 root root 435893187 Jun 15 23:03 mpnet_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny MPNet model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import MPNetEmbeddings
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

MPNet_loaded = MPNetEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("MPNet")

pipeline = Pipeline(stages=[
    document_assembler,
    MPNet_loaded
])

data = spark.createDataFrame([[
    "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."
]]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(MPNet.embeddings) as embeddings").show()

+--------------------+
|          embeddings|
+--------------------+
|[-0.020282406, 0....|
+--------------------+



That's it! You can now go wild and use hundreds of MPNet models from HuggingFace 🤗 in Spark NLP 🚀
